# Project 2: Span Identification with Sequence Labeling Models
## CS4740/5740 Fall 2020

### Project Submission Due: Oct 23rd
Please submit **pdf file** of this notebook on **Gradescope**, and **ipynb** on **CMS**. For instructions on generating pdf and ipynb files, please refer to project 1 instructions.



Names: Yiqing Luo, Zehua Zhang

Netids: yl2546, zz727


Don't forget to share your newly copied notebook with your partner!


**Reminder: both of you can't work in this notebook at the same time from different computers/browser windows because of sync issues. We even suggest to close the tab with this notebook when you are not working on it so your partner doesn't get sync issues.**


### Q0: Individual Member Contribution

Briefly explain the contribution of individual group members here. Report if working loads are unfairly distributed.

# Overview

---

In this project, you will implement a model that identifies relevant information in a text and tags it with the appropriate label. Particularly, the task of this project is **Propaganda Identification**. The given dataset contains (manual) annotations indicating fragments of text that exhibit one of a set of well-known [propaganda techniques](https://propaganda.qcri.org/annotations/definitions.html). Your task is to develop NLP models to identify these propagandistic spans of text automatically. We will treat this as a **sequence-tagging task**: for each token in the input text, assign a label $y\in\{0,1\}$, such that *1 represents propagandistic text* and *0 represents non-propaganda*.   (A description of the original task formulation is [here](https://propaganda.qcri.org/ptc/).  We are working on a modified version of their "span identification" task.)

For this project, you will implement two sequence labeling approaches:

- Model 1 : a Hidden Markov Model (HMM)
- Model 2 : a Maximum Entropy Markov Model (MEMM), which is an adaptation of an HMM in which a Logistic Regression classifier (also known as a MaxEnt classifier) is used to obtain the lexical generation probabilities (i.e., the observation/emission probability matrix, so "observations" == "emissions" == "lexical generations").  Feature engineering is strongly suggested. (Papers from the [Workshops on Figurative Language Processing](https://sites.google.com/view/figlang2020/) can provide good insights on feature selection for this task.) You can also refer to the J&M book. 

Implementation of the Viterbi algorithm (for finding the most likely tag sequence to assign to an input text) is required for both parts, so make sure that you understand it ASAP.

You will implement and train two sequence tagging models, generate your predictions for the provided test set, and submit them to **Kaggle**. Please enter all code and answer the questions of this colab notebook.

**Jurafsky & Martin reading on HMMs and MEMMs can be found in Ch. 8.3–8.5.** The code you write can be added anywhere in the document, but we implore you to keep it readable. You will be asked to describe and motivate your models in parts of the document. You will be graded on both the code and text you write; see grading details at the end of the document.


# Notes

---

1. Please read through the entire notebook before you start coding. That might inform your code structure.
2. Grading breakdown is found at the end; please consult it.
3. Google colab does **not** provide good synchronization; we do not recommend multiple people to work on the same notebook at the same time.
4. The project is somewhat open ended. ("But that's a good thing.  Really. It's more fun that way", says Claire and Esin.) We will ask you to implement some model, but precise data structures and so on can be chosen by you. However, to integrate with Kaggle, you will need to submit Kaggle predictions using our tokenization code.  As a result, **it is probably easiest if you use our tokenization code for the entire project**.
5. You will be asked to fill in your code at various points of the document. You will also be asked to answer questions that analyze your results and motivate your implementation. These questions are named Q1-Q8. You may create additional cells to insert code, text and images as necessary.
6. Kaggle is not able to calculate *span-level* P/R/F1 measures, which is the standard way to evaluate this type of sequence-tagging task. And we don't actually care so much about the token-level tagging accuracy, which Kaggle **can** calculate.  In particular, there are many fewer propaganda tokens than non-propaganda ones, so always guessing "non-propaganda" would produce a very high accuracy.  So we are compromising by using token-level **weighted accuracy**.  Here is how it works:

A **weighted accuracy** metric that favors finding propagandistic tokens over non-propagandistic ones. The weights for both classes are the inverse of their frequencies.  

``` 
frac_propaganda = num_propaganda/num_labels   [in the answer key]
weight_propaganda = 1/frac_propaganda  
weight_non_propaganda = 1/(1-frac_propaganda)

weighted_accuracy = 
   ((weight_propaganda * # propaganda correct) 
                      +
   (weight_non_propaganda * # non_propaganda correct)) / 
   
   ((weight_propaganda * num_propaganda) 
                      +
   (weight_non_propaganda * num_non_propaganda))
```  
This is also known as the **macro average**, i.e., the average of the accuracy for each label type.


# Task and dataset

---

1. Obtain the data from Kaggle at https://www.kaggle.com/t/8a8030baefcc4d91b715f114353dba38.
2. Unzip the data. Put it into your google drive, and mount it on colab as per below:

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)



train_path = os.path.join(os.getcwd(), "drive", "My Drive/CS4740/project2", "train") # replace based on your Google drive organization
test_path = os.path.join(os.getcwd(), "drive", "My Drive/CS4740/project2", "test") # replace based on your Google drive organization
print(os.listdir(train_path)[:5])
print(os.listdir(test_path)[:5])

Mounted at /content/drive
['article725731328.task-si.labels', 'article702077783.txt', 'article724095467.txt', 'article776535164.txt', 'article763761219.txt']
['article741655444.txt', 'article755170235.txt', 'article727634031.txt', 'article769427494.txt', 'article758512204.txt']


3. The *train* directory contains *article{XXX}.txt* files which are plaintext documents and also *label* files such as *article{XXX}.task-si.labels*. These label files correspond to the byte-span offests of each segment of propaganda in the associated article. (The tokenizer that we describe just below converts the byte-span representation of propagandistic text spans  into the token-level gold-standard labels that your sequence-tagging models require for training.) The test directory *only* contains articles; you will use your models to detect the propagandistic spans within them.  

4. We provide a tokenizer for these documents. You **must** use this tokenizer as the labels that Kaggle expects are based upon this tokenization. The code below tokenizes each document and generates the appropriate token-level labels consistent with the associated *labels* file. (This is so that you do not need to perform any byte-level text processing.  In particular, the tokenizer  merges nested or overlapping propagandistic text spans from the article into a single segment. You really shouldn't have to look at the *lables* files at all.) The code uses python type annotations; these indicate the type of arguments the functions take.

5. Documents are represented as a list of strings, each being a token. Labels are represented as a list of integers in {0,1}, 1 corresponding to a propagandistic token and 0 to not propaganda.



In [ ]:
import os
from typing import List, Tuple
import numpy as np
import pandas as pd


from nltk import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def read_txt(fname):
  with open(fname) as open_article:
    lines = open_article.read()
  return lines

def read_labels(labels : str) -> List[Tuple[int, int]]:
	"processing of labels file"
	labels = labels.split("\n")[:-1]
	labels = [tuple(map(int, l.split("\t")[1:])) for l in labels]
	return labels

def sort_and_merge_labels(labels : List[Tuple[int, int]]) -> List[Tuple[int, int]]:
  "sort labels, necessary for later splitting"
  if len(labels) == 0:
    return labels
  labels = list(sorted(labels, key = lambda t: t[0]))
  # merge
  curr = labels[0]
  merged = []
  for l in labels[1:]:
      # if distinct, add
      if l[0] > curr[1]:
        merged.append(curr)
        curr = l
      # else merge
      else:
        curr = (curr[0], max(curr[1], l[1]))
  merged.append(curr)
  return merged

def split_with_labels(labels : List[Tuple[int, int]], article : str) -> Tuple[List[str], List[int]]:
  "split text into segments based upon labels"
  if len(labels) == 0:
    return [article], [0]
  segments = []
  binary_class = []
  start = 0
  for l_start, l_end in labels:
    std_seg = article[start:l_start]
    prop_seg = article[l_start:l_end]
    segments.append(std_seg)
    binary_class.append(0)
    segments.append(prop_seg)
    binary_class.append(1)
    start = l_end
  last_seg = article[start:]
  segments.append(last_seg)
  binary_class.append(0)
  return segments, binary_class

def remove_newline_fix_punc_seg(segments):
  " preprocessing necessry for tokenization to be consistent"
  segments = [s.replace("\n", " ").replace(".", " .") for s in segments]
  return segments

def remove_newline_fix_punc_art(article):
  " preprocessing necessry for tokenization to be consistent"
  article = article.replace("\n", " ").replace(".", " .")
  return article

def get_toks(input):
  output = []
  for toks in [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(input)]:
    output += toks
  return output

# This is the function you may need to call
def tokenize_article(article_file):
  "calls all functions above and perform sanity checks"
  article = read_txt(article_file)
  article = remove_newline_fix_punc_art(article)
  art_toks = get_toks(article)
  return art_toks

# This is the function you may need to call
def master_tokenizer(article_file, labels_file):
  "calls all functions above and perform sanity checks"
	# read and get labels
  article = read_txt(article_file)
  labels = read_txt(labels_file)
  labels = read_labels(labels)
  labels = sort_and_merge_labels(labels)
  segments, binary_class = split_with_labels(labels, article)
  article = remove_newline_fix_punc_art(article)
  segments = remove_newline_fix_punc_seg(segments)
  # sanity check
  reconstructed = ""
  for seg, lab in zip(segments, binary_class):
    reconstructed += seg
  assert reconstructed == article
	# tokenize
  seg_toks = []
  new_labels = []
  for seg, label in zip(segments, binary_class):
    new_toks = get_toks(seg)
    seg_toks += new_toks
    new_labels += [label for _ in range(len(new_toks))]
	# sanity check
  art_toks = get_toks(article)
  sanity = True
  if len(art_toks) != len(seg_toks):
    sanity = False
  for i, (at, st, lab) in enumerate(zip(art_toks, seg_toks, new_labels)):
    if at != st:
      sanity = False
      break
  return seg_toks, new_labels, sanity




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


 6. Execute the commands below to visualize the tokenization:

In [ ]:
article_file = "article698018235.txt"
labels_file = "article698018235.task-si.labels"
article_file = os.path.join(train_path, article_file)
labels_file = os.path.join(train_path, labels_file)
tokens, labels, _ = master_tokenizer(article_file, labels_file)

print(len(tokens), len(labels), tokens[:10])

700 700 ['dan', 'fishback', ':', 'it', "'s", 'okay', 'to', 'boycott', 'israeli', 'plays']


7.  Provide some quantitative data exploration. Assess dataset size, documents lengths and class inbalance. Give some examples of sentences containing propaganda techniques.

In [ ]:
#examples of sentences containing propaganda techniques
start_end=[]
prev=0

for i in range(len(labels)): #find occurances of "1" and record index
  cur=labels[i]
  if cur==1 and prev==0:
    start=i
  if cur==0 and prev==1:
    end=i
    start_end.append((start,end))
  prev=cur
    
prop_example=[]
for tups in start_end: #use the indexes in labels to find tokens and build sentences
  start=tups[0]
  end=tups[1]
  sentence=" ".join(tokens[start:end])
  prop_example.append(sentence)

prop_example[:10]

['radical anti-israel hate group',
 'anti-israel bds hate group',
 'anti-semite',
 'accused jews of drinking blood',
 "eu no longer considers # hamas a terrorist group . time for us to do same . ''",
 'anti-israel',
 "any jew who opposes the occupation — or opposes zionism itself — knows that feeling of being shunned from the places that are supposed to shelter and nurture you : families , synagogues , community centers , arts organizations , ''",
 'terrified',
 'if our jewish institutions — particularly the american jewish historical society — can not accommodate dissent , and effectively exclude all jewish anti-zionists , then they have not only lost a rapidly growing jewish population , but they have lost a key aspect of their jewishness',
 'knapsack of entitled nonsense']

In [ ]:
# Collect all the data
article_files = [file for file in os.listdir(train_path) if file[-3:] == 'txt']
labels_files = [file for file in os.listdir(train_path) if file[-6:] == 'labels']
article_files.sort()
labels_files.sort()
print('Missing data or label' if len(article_files) != len(labels_files) else None)

total_data = []
total_labels = []
for article_file, labels_file in zip(article_files, labels_files):
  article_file = os.path.join(train_path, article_file)
  labels_file = os.path.join(train_path, labels_file)
  data, labels, sanity = master_tokenizer(article_file, labels_file)
  if not sanity:
    print(article_file)
    continue
  total_data.append(data)
  total_labels.append(labels)

# Total articles in train dataset:
print("Total articles in train dataset: ", len(article_files))
print()

# Average tokens in an article: 
print("Average tokens in an article: ", 
   np.mean([len(file) for file in total_data]))
print()

# Count of propagandistic token and count of nonpropagandistic token: 
print("Count of propagandistic token: ",
   np.sum([file.count(1) for file in total_labels]))
print("Count of nonpropagandistic token: ",
   np.sum([file.count(0) for file in total_labels]))
print("Classes are imbalance: propagandistic token is far less than nonpropagandistic token")
print()

### Q1: Initial data observations
What are your initial observations of the dataset after you explore the dataset?

**Answer:** There is a total of 260 articles in the training dataset. The average number of tokens in an article is 966. The count of propagandistic token is 26497 and the count of nonpropagandistic token is 224778. The classes are imbalanced since propagandistic token is far less than nonpropagandistic token.

# Model 1: HMM Implementation

---

In this section, you will implement a HMM model for this task. We expect:


1. An implementation of the **Viterbi algorithm** that can be used to infer token-level labels --- propaganda or not propaganda --- for an input document.   This process is commonly referred to as **decoding**.
2. Code for counting and smoothing of labels and words as necessary to support the HMM decoding. (This is pretty much what you already know how to do from project 1.)


The tokenization of documents can be performed with the code we provide above. We suggest you calculate probabilities in a log form.  Bigram Viterbi is O(sm^2) where s is the length of the sentence and m is the number of tags. Your implementation should have similar efficiency.

Code of Academic Integrity:  We encourage collaboration regarding ideas, etc. However, please **do not copy code from online or share code with other students**. We will be running programmes to detect plagiarism.

In [ ]:
# Your implementation here
# we expect a function or class, mapping a sequence of tokens to a sequence of labels
# this function or class will be called below
def hmm_train(data, labels, k):
  '''
  Train the data to get the transition probability matrix, 
  the emission probabilities, the initial probability distribution. 
  We have already known that there are only two state: 
  propagandistic (1) and propaganda (0). 
  Input:
  data: list of list of tokens
  label: list of list of {0, 1}
  k: integer for add-k smoothing
  Output:
  tran_prob: matrix of shape (2, 2) represent log transition probability
    tran_prob[i][j] means P(t = j|t = i)
  emi_prob: dictionary of list represent log emission probability
    emi_prob[token][i] means P(w = token| t = i)
  init_prob: matrix of shape (2, ) represent log initial probability
  unk_porb: matrix of shape (2, ) represent log unknown word emission probability
  '''

  tran_prob = np.zeros((2, 2))
  emi_prob = {}
  init_prob = np.zeros(2)
  unk_prob = np.zeros(2)
  for i in range(len(data)):
    unk_prob[0] += labels[i].count(0)
    unk_prob[1] += labels[i].count(1)
    init_prob[labels[i][0]] += 1
    if data[i][0] not in emi_prob:
      emi_prob[data[i][0]] = [0, 0]
    emi_prob[data[i][0]][labels[i][0]] += 1
    for j in range(len(data[i]) - 1):
      tran_prob[labels[i][j]][labels[i][j + 1]] += 1
      if data[i][j] not in emi_prob:
        emi_prob[data[i][j]] = [0, 0]
      emi_prob[data[i][j]][labels[i][j]] += 1
  for i in range(2):
    tran_prob[i] = np.log(tran_prob[i] / unk_prob[i])

  unk_prob += k * len(emi_prob)
  for token in emi_prob.keys():    
    for i in range(2):
      emi_prob[token][i] = np.log((emi_prob[token][i] + k) / unk_prob[i])

  init_prob = np.log(init_prob / len(data))
  unk_prob = np.log(unk_prob / np.sum(unk_prob))
  return tran_prob, emi_prob, init_prob, unk_prob

def handle_oov(word):
  if word in emi_prob:
    return word
  else:
    for i in range(len(word)):
      for w in emi_prob.keys():
        if w.startswith(word[:-i]):
          return w
    return 'a'

def hmm_predict(data, tran_prob, emi_prob, init_prob, unk_prob):
  '''
  Predict the labels of data according to the training result
  Input:
  data: list of list of tokens
  tran_prob: matrix of shape (2, 2) represent transition probability
    tran_prob[i][j] means P(t = j|t = i)
  emi_prob: dictionary of list represent emission probability
    emi_prob[token][i] means P(w = token| t = i)
  init_prob: matrix of shape (2, ) represent initial probability
  unk_porb: matrix of shape (2, ) represent unknown word emission probability
  Output:
  label: list of list of {0, 1}
  '''
  prediction = []
  for tokens in data:
    length = len(tokens)
    score = np.zeros((2, length))
    bptr = np.zeros((2, length))
    word = handle_oov(tokens[0])
    for i in range(2):
      score[i][0] = init_prob[i] + emi_prob[word][i]
    for t in range(1, length):
      word = handle_oov(tokens[t])
      for i in range(2):
        temp = [score[j][t - 1] + tran_prob[j][i] for j in range(2)]
        score[i][t] = max(temp) + emi_prob[word][i]
        bptr[i][t] = np.argmax(temp)
    T = np.zeros(length).astype(int)
    T[-1] = np.argmax(score[:, -1])
    for i in range(length - 2, -1, -1):
      T[i] = bptr[T[i + 1]][i + 1]
    prediction.append(T)
  
  return prediction


In [ ]:
#a function that takes in a 1-d list of tokens to output a 1-d list of labels for the submission generater
def hmm(data):
  test=[]
  test.append(data)
  tran_prob, emi_prob, init_prob, unk_prob = hmm_train(total_data, total_labels, 0.01)
  hmm_prediction = hmm_predict(test, tran_prob, emi_prob, init_prob, unk_prob)
  label_sequence=hmm_prediction[0].tolist()
  return label_sequence

## Validation Step

1. Create a validation set from the given dataset, i.e. a subset of (~10%) the training dataset that you only use for evaluating the models, not for training.  (You can think of the validation set as a sample test set.)
2. Train your HMM model on the (remainder of the) training set and evaluate it on the validation set. Report **weighted accuracy**, which is explained in the *Notes* section above.

Please also take a look into your misclassified cases, as we will be performing error analysis in the *Evaluation* section. We expect smoothing, unknown word handling and correct emission (i.e., lexical generaion) probabilities. 

In the *Kaggle Submission* section, there is code provided for generating the output file in the form required for Kaggle.  If you find it useful for computing weighted accuracy, you can use it here as well.

In [ ]:
# Evaluate/validate your model here
# you may attach pictures of graphs etc.

import random

def accuracy(prediction, labels):
  '''
  Calculate the accuracy given the prediction and ground truth
  Input:
  predition: list of list of {0, 1}  
  label: list of list of {0, 1}
  Output:
  accuracy: float between 0 and 1
  '''

  propaganda = 0
  propaganda_correct = 0
  non_propaganda = 0
  non_propaganda_correct = 0
  for pred, label in zip(prediction, labels):
    for p, l in zip(pred, label):
      if l == 1:
        propaganda += 1
        if p == 1:
          propaganda_correct += 1
      else:
        non_propaganda += 1
        if p == 0:
          non_propaganda_correct += 1
  # print((propaganda_correct / propaganda), (non_propaganda_correct / non_propaganda))

  return ((propaganda_correct / propaganda) + (non_propaganda_correct / non_propaganda)) / 2


In [ ]:
#create training data and validation data
total_combine = list(zip(total_data, total_labels))
random.shuffle(total_combine)
shuffled_data, shuffled_labels = zip(*total_combine)

validation_data = shuffled_data[:26]
validation_labels = shuffled_labels[:26]
training_data = shuffled_data[26:]
training_labels = shuffled_labels[26:]

In [ ]:
tran_prob, emi_prob, init_prob, unk_prob = hmm_train(training_data, training_labels, 0.01)
hmm_prediction = hmm_predict(validation_data, tran_prob, emi_prob, init_prob, unk_prob)
weighted_accuracy = accuracy(hmm_prediction, validation_labels)
print(weighted_accuracy)

0.5542016358227231




### Q2 : Explan your HMM implementations

Q2.1: Explain here how you implemented HMMs (e.g. **which algorithms/data structures** you used). Make clear which parts were implemented from scratch vs. obtained via an existing package. 

**Answer:** Every part of this HMM are implemented from scratch. We use matrix/list for transition probability, initial probability. We use dictionary for emission probability. Because we want to find the emission probability of a word. So we use dictionary and the keys are words. For unknown words handling, we use prefix to find the similar word in the emission probability dictionary. We also implement Viterbi algorithms from scratch. 

Q2.2: Explain and motivate any design choices providing the intuition behind them (e.g. which methods you used for your HMM implementation, why?).

**Answer:** There are several design we use. First is Viterbi algorithms for decoding. Because we need to reduce the search complexity for decoding, we use Viterbi algorithms. We use prefix to handle unknown words. We believe similar words might have similar meanings and they are more likely to be both propagandistic of both not. And the prefix affect the meaning of a word more than suffix. So if there is an unknown word, we choose the word from the dictionary which had the longest common prefix to explain the unknown word. We also use add-k smoothing. Because we will have situations which never happened in the training, like a word have a new state. To prevent 0 in log, we use add-k smoothing. And we don't want the add-k to affect the accuracy, we did experiment for different k values and determine the k value based on accuracy. 



In [ ]:
def gen_hmm_accuracies(klist):
  accuracies=[]
  for i in range(len(klist)):
    tran_prob, emi_prob, init_prob, unk_prob = hmm_train(training_data, training_labels, klist[i])
    prediction = hmm_predict(validation_data, tran_prob, emi_prob, init_prob, unk_prob)
    weighted_accuracy = accuracy(prediction, validation_labels)
    accuracies.append(weighted_accuracy)
  return accuracies


In [ ]:
klist=[1, 0.1, 0.01, 0.001]
accuracies=gen_hmm_accuracies(klist)
rowname=["k="+str(klist[i]) for i in range(len(klist))]
pd.DataFrame(accuracies,rowname,["Weighted Accruacy"])

,Weighted Accruacy
k=1,0.499738
k=0.1,0.540672
k=0.01,0.554202
k=0.001,0.544219


In [ ]:
def accuracy_df(prediction, labels):
  falseneg = 0
  truepos = 0
  falsepos = 0
  trueneg = 0
  for pred, label in zip(prediction, labels):
    for p, l in zip(pred, label):
      if l == 1 and p == 1: 
        truepos += 1
      if l==1 and p==0:
        falseneg+=1
      if l==0 and p == 0:
        trueneg += 1
      if l==0 and p==1:
        falsepos+=1
  return (truepos,falseneg,trueneg,falsepos)

In [ ]:
hmm_true_pos,hmm_false_neg,hmm_true_neg,hmm_false_pos=accuracy_df(hmm_prediction,validation_labels)
hmm_df=pd.DataFrame([[hmm_true_neg,hmm_false_neg], [hmm_false_pos, hmm_true_pos]],
                   columns=['non-propaganda', 'propaganda'])
hmm_df

,non-propaganda,propaganda
0,20313,1812
1,716,301


In [ ]:
percent_true_pos=hmm_true_pos/(hmm_false_neg+hmm_true_pos)
percent_true_neg=hmm_true_neg/(hmm_false_pos+hmm_true_neg)
print("Percentage of propaganda correctly labeled:",percent_true_pos)
print("Percentage of nonpropaganda correctly labeled:",percent_true_neg)

Percentage of propaganda correctly labeled: 0.14245149077141506
Percentage of nonpropaganda correctly labeled: 0.9659517808740311


In [ ]:
#transition probabilities
transition=[0,1]
pd.DataFrame(np.exp(tran_prob),transition,transition)

,0,1
0,0.985237,0.013634
1,0.114419,0.885417


### Q3: Results Analysis

Q3.1: Explain here how you evaluated the models. Summarize the performance of your system and any variations that you experimented with on the validation datasets. Put the results into clearly labeled tables or diagrams and include your observations and analysis. 

**Answer:** We evaluted the performance of the model using weighted accuracy. We used add-k smoothing in our HMM model and experimented with different k values. The above table showed that a we get highest weighted accuracy when k=0.01, so we picked it as our parameter value.


Q3.2: When did the system work well? When did it fail?  Any ideas as to why? How might you improve the system?

**Answer:** As the above table shows, our model predicts nonpropagandistic words much better than propagandistic words. A possible reason may be that, according to the table above, transition probability 0->0 is much higher than transition probability 0->1, and transition probability 1->1 is much higher than that of 1->0. When a propaganda word has previous tag=0, it is more likely to be predicted to have label 0 rather than 1, and the training data has much more nonpropagandistic words than propagandistic words. The system can be improved by taking in a larger window of tokens and tags into consideration.

Q3.3: What is the effect of unknown word handling and smoothing?

**Answer:** Unknown word handling deals with word in test data that has never occurred during training. Smoothing gives a non-zero probability to word-label pair that was not seen during training, to avoid log(0). Both of them increase accuracy.



# Model 2: MEMM Implementation

---


In this section, you will implement a Maximum Entropy Markov Model (**MEMM**) to perform the same propaganda detection task. Your model should consist of a MaxEnt classifier with Viterbi decoding. 
 
1. We have already performed tokenizations for documents. You can either use a MaxEnt classifier from an existing package or write the MaxEnt code yourself. **Important note:  MaxEnt classifiers are statistically equivalent to multi-class logistic regression, so you can use packages for multi-class LR instead of MaxEnt.**

2. Use the classifier to learn a probability P(t_i|features). You may replace either the lexical generation probability – P(w_i|t_i) – or the transition probability – P(t_i|t_{i−1}) – in the HMM with it, or you may replace the entire *lexical generation probability * transition probability*  calculation – P (w_i|t_i) ∗ P (t_i|t_{i−1)} –  in the HMM with it. 

3. To train such classifier, you need to pick some feature set. The content of the feature set is up to your choice. You should be trying different feature sets, and evaluate your choices on the validation set. Pick the feature set that performs overall the best according to the F1 measure.

4. Use your own implementation of the **Viterbi algorithm**, which you can modify from the one you developed for the HMM model. You will need the probabilities that you obtain from the MaxEnt classifier. 

5. Remember to use same training and validation split when evaluating the MEMM to have a **fair comparison** with your **HMM model**.


Please also take a look into your misclassified cases, as we will be performing error analysis in *Evaluation* section. 





---
Work flow summary:

![alt text](https://drive.google.com/uc?export=view&id=14VfjW3yDyXLojWM_u0LeJYdDOSLkElBn)




In [ ]:
# Your model implementation here
# we expect a function of class, mapping a sequence of tokens to a sequence of labels
# this function or class will be called below
#
# You will need:
# 1. Extract Features
# 2. Train MaxEnt
# 3. To call Viterbi 


In [ ]:
actual_labels=[]
for article in training_labels:
  for i in range(len(article)):
    actual_labels.append(article[i])

In [ ]:
#a list of current word
current_word=[]
for article in training_data:
  for i in range(len(article)):
    current_word.append(article[i])

In [ ]:
#a list of previous tags
previous_tag=[]
for article in training_labels:
  previous_tag.append(-1) #word at start of sentence has previous tag -1
  for i in range(1,len(article)):
    previous_tag.append(article[i-1])

In [ ]:
#a list of part of speech tags
training_pos=[]
for article in training_data:
  pos_output=nltk.pos_tag(article)
  for tup in pos_output:
    training_pos.append(tup[1])

In [ ]:
#a list of previous words, with start token at beginning of each article
previous_word=[]
for article in training_data:
  previous_word.append('<s>')
  for i in range(1,len(article)):
    previous_word.append(article[i])

In [ ]:
#generate feature dictionary
word_tag_dic=[{'cur_word': word, 'prev_tag': prev_tag} for word,prev_tag in zip(current_word,previous_tag)]
word_pos_dic=[{'cur_word': word, 'pos':pos_tag} for word,pos_tag in zip(current_word,training_pos)]
cur_pre_word_pos_dic=[{'cur_word': word, 'pos':pos_tag, 'pre_word':pre_word} for word,pos_tag,pre_word in zip(current_word,training_pos,previous_word)]
word_tag_pos_dic=[{'cur_word': word, 'prev_tag': prev_tag,'pos':pos_tag} for word,prev_tag,pos_tag in zip(current_word,previous_tag,training_pos)]
all_features_dic=[{'cur_word': word, 'prev_tag': prev_tag,'pos':pos_tag,'pre_word':pre_word} for word,prev_tag,pos_tag,pre_word in zip(current_word,previous_tag,training_pos,previous_word)]

In [ ]:
#generate feature set input
def featureset(dic_list,labels):
  featureset=[]
  for i in range(len(labels)):
    featureset.append((dic_list[i],str(labels[i])))
  return featureset

In [ ]:
features_word_tag=featureset(word_tag_dic,actual_labels)
features_word_pos=featureset(word_pos_dic,actual_labels)
features_cur_pre_word_pos=featureset(cur_pre_word_pos_dic,actual_labels)
features_word_tag_pos=featureset(word_tag_pos_dic,actual_labels)
all_features=featureset(all_features_dic,actual_labels)

In [ ]:
#train classifiers
from nltk.classify import MaxentClassifier
classifier_word_tag = MaxentClassifier.train(features_word_tag,max_iter=2)

  ==> Training (2 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.107
         Final          -0.08939        0.952


In [ ]:
classifier_word_tag_pos = MaxentClassifier.train(features_word_tag_pos,max_iter=5)

  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.107
             2          -0.12747        0.902
             3          -0.08899        0.974
             4          -0.06826        0.978
         Final          -0.05735        0.978


In [ ]:
classifier_word_pos = MaxentClassifier.train(features_word_pos,max_iter=5)

  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.107
             2          -0.20996        0.893
             3          -0.20468        0.895
             4          -0.20030        0.901
         Final          -0.19711        0.903


In [ ]:
classifier_cur_pre_word_pos = MaxentClassifier.train(features_cur_pre_word_pos,max_iter=5)

  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.107
             2          -0.21287        0.894
             3          -0.20598        0.903
             4          -0.20098        0.903
         Final          -0.19755        0.903


In [ ]:
classifier_all = MaxentClassifier.train(all_features,max_iter=5)

  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.107
             2          -0.13944        0.902
             3          -0.10474        0.944
             4          -0.08167        0.977
         Final          -0.06762        0.979


In [ ]:
# Run your model on validation set
# You will need to 
# 1. Call your function above to get a prediction result on Validation Set
# 2. Report Metrics
# (See if you need to modify your feature set)

In [ ]:
#current word and previous tag
def memm_predict_word_tag(data, emi_prob, classifier):
  '''
  Predict the labels of data according to the training result
  Input:
  data: list of list of tokens
  emi_prob: dictionary of list represent emission probability
    emi_prob[token][i] means P(w = token| t = i)
  Output:
  label: list of list of {0, 1}
  '''
  prediction = []
  for tokens in data:
    length = len(tokens)
    score = np.zeros((2, length))
    bptr = np.zeros((2, length))
    probdict=classifier.prob_classify({'word': tokens[0], 'prevtag': -1})
    for i in range(2):
      classifier_prob = probdict.prob(str(i))
      score[i][0] = np.log(classifier_prob)
    for t in range(1, length):
      word = tokens[t]
      for i in range(2):
        temp = []
        for j in range(2):
          probdict=classifier.prob_classify({'word': tokens[t], 'prevtag': j})
          classifier_prob = probdict.prob(str(i))
          temp.append(score[j][t-1] + np.log(classifier_prob))
        score[i][t] = max(temp)
        bptr[i][t] = np.argmax(temp)
    T = np.zeros(length).astype(int)
    T[-1] = np.argmax(score[:, -1])
    for i in range(length - 2, -1, -1):
      T[i] = bptr[T[i + 1]][i + 1]
    prediction.append(T)
  
  return prediction

In [ ]:
#current word, previous tag and part of speech
def memm_predict_word_tag_pos(data, emi_prob, classifier):
  '''
  Predict the labels of data according to the training result
  Input:
  data: list of list of tokens
  emi_prob: dictionary of list represent emission probability
    emi_prob[token][i] means P(w = token| t = i)
  Output:
  label: list of list of {0, 1}
  '''

  #create pos list
  pos=[]
  for i in range(len(data)):
    pos.append([])
    pos_output=nltk.pos_tag(data[i])
    for tup in pos_output:
      pos[i].append(tup[1])

  prediction = []
  for index in range(len(data)):
    tokens=data[index]
    article_pos=pos[index]
    length = len(tokens)
    score = np.zeros((2, length))
    bptr = np.zeros((2, length))
    probdict=classifier.prob_classify({'word': tokens[0], 'pos':article_pos[0],'prev_tag':-1})
    for i in range(2):
      classifier_prob = probdict.prob(str(i))
      score[i][0] = np.log(classifier_prob)
    for t in range(1, length):
      word = tokens[t]
      for i in range(2):
        temp = []
        for j in range(2):
          probdict=classifier.prob_classify({'word': tokens[t], 'pos':article_pos[t],'prev_tag':j})
          classifier_prob = probdict.prob(str(i))
          temp.append(score[j][t-1] + np.log(classifier_prob))
        score[i][t] = max(temp)
        bptr[i][t] = np.argmax(temp)
    T = np.zeros(length).astype(int)
    T[-1] = np.argmax(score[:, -1])
    for i in range(length - 2, -1, -1):
      T[i] = bptr[T[i + 1]][i + 1]
    prediction.append(T)
  
  return prediction

In [ ]:
#current word and part of speech
def memm_predict_word_pos(data, emi_prob, classifier):
  '''
  Predict the labels of data according to the training result
  Input:
  data: list of list of tokens
  emi_prob: dictionary of list represent emission probability
    emi_prob[token][i] means P(w = token| t = i)
  Output:
  label: list of list of {0, 1}
  '''

  #create pos list
  pos=[]
  for i in range(len(data)):
    pos.append([])
    pos_output=nltk.pos_tag(data[i])
    for tup in pos_output:
      pos[i].append(tup[1])

  prediction = []
  for index in range(len(data)):
    tokens=data[index]
    article_pos=pos[index]
    length = len(tokens)
    score = np.zeros((2, length))
    bptr = np.zeros((2, length))
    probdict=classifier.prob_classify({'word': tokens[0], 'pos':article_pos[0]})
    for i in range(2):
      classifier_prob = probdict.prob(str(i))
      score[i][0] = np.log(classifier_prob)
    for t in range(1, length):
      word = tokens[t]
      for i in range(2):
        temp = []
        for j in range(2):
          probdict=classifier.prob_classify({'word': tokens[t], 'pos':article_pos[t]})
          classifier_prob = probdict.prob(str(i))
          temp.append(score[j][t-1] + np.log(classifier_prob))
        score[i][t] = max(temp)
        bptr[i][t] = np.argmax(temp)
    T = np.zeros(length).astype(int)
    T[-1] = np.argmax(score[:, -1])
    for i in range(length - 2, -1, -1):
      T[i] = bptr[T[i + 1]][i + 1]
    prediction.append(T)
  
  return prediction

In [ ]:
#current word, previous word and part speech
def memm_predict_cur_pre_word_pos(data, emi_prob, classifier):
  '''
  Predict the labels of data according to the training result
  Input:
  data: list of list of tokens
  emi_prob: dictionary of list represent emission probability
    emi_prob[token][i] means P(w = token| t = i)
  Output:
  label: list of list of {0, 1}
  '''

  #create pos list
  pos=[]
  for i in range(len(data)):
    pos.append([])
    pos_output=nltk.pos_tag(data[i])
    for tup in pos_output:
      pos[i].append(tup[1])

  prediction = []
  for index in range(len(data)):
    tokens=data[index]
    article_pos=pos[index]
    length = len(tokens)
    score = np.zeros((2, length))
    bptr = np.zeros((2, length))
    probdict=classifier.prob_classify({'word': tokens[0], 'pos':article_pos[0],'pre_word':'<s>'})
    for i in range(2):
      classifier_prob = probdict.prob(str(i))
      score[i][0] = np.log(classifier_prob)
    for t in range(1, length):
      word = tokens[t]
      for i in range(2):
        temp = []
        for j in range(2):
          probdict=classifier.prob_classify({'word': tokens[t], 'pos':article_pos[t],'pre_word':tokens[t-1]})
          classifier_prob = probdict.prob(str(i))
          temp.append(score[j][t-1] + np.log(classifier_prob))
        score[i][t] = max(temp)
        bptr[i][t] = np.argmax(temp)
    T = np.zeros(length).astype(int)
    T[-1] = np.argmax(score[:, -1])
    for i in range(length - 2, -1, -1):
      T[i] = bptr[T[i + 1]][i + 1]
    prediction.append(T)
  
  return prediction

In [ ]:
#current word, previous word and part speech
def memm_predict_all(data, emi_prob, classifier):
  '''
  Predict the labels of data according to the training result
  Input:
  data: list of list of tokens
  emi_prob: dictionary of list represent emission probability
    emi_prob[token][i] means P(w = token| t = i)
  Output:
  label: list of list of {0, 1}
  '''

  #create pos list
  pos=[]
  for i in range(len(data)):
    pos.append([])
    pos_output=nltk.pos_tag(data[i])
    for tup in pos_output:
      pos[i].append(tup[1])

  prediction = []
  for index in range(len(data)):
    tokens=data[index]
    article_pos=pos[index]
    length = len(tokens)
    score = np.zeros((2, length))
    bptr = np.zeros((2, length))
    probdict=classifier.prob_classify({'word': tokens[0], 'pos':article_pos[0],'pre_word':'<s>','pre_tag':-1})
    for i in range(2):
      classifier_prob = probdict.prob(str(i))
      score[i][0] = np.log(classifier_prob)
    for t in range(1, length):
      word = tokens[t]
      for i in range(2):
        temp = []
        for j in range(2):
          probdict=classifier.prob_classify({'word': tokens[t], 'pos':article_pos[t],'pre_word':tokens[t-1],'pre_tag':j})
          classifier_prob = probdict.prob(str(i))
          temp.append(score[j][t-1] + np.log(classifier_prob))
        score[i][t] = max(temp)
        bptr[i][t] = np.argmax(temp)
    T = np.zeros(length).astype(int)
    T[-1] = np.argmax(score[:, -1])
    for i in range(length - 2, -1, -1):
      T[i] = bptr[T[i + 1]][i + 1]
    prediction.append(T)
  
  return prediction

In [ ]:
#current word and previous tag
memm_prediction_word_tag=memm_predict_word_tag(validation_data,emi_prob,classifier_word_tag)
weighted_accuracy = accuracy(memm_prediction_word_tag, validation_labels)
weighted_accuracy

0.5

In [ ]:
#current word and previous tag
memm_true_pos,memm_false_neg,memm_true_neg,memm_false_pos=accuracy_df(memm_prediction_word_tag,validation_labels)
memm_df=pd.DataFrame([[memm_true_neg,memm_false_neg], [memm_false_pos, memm_true_pos]],
                   columns=['non-propaganda', 'propaganda'])
memm_df

,non-propaganda,propaganda
0,21029,2113
1,0,0


In [ ]:
#current word, previous tag and part of speech
memm_prediction_word_tag_pos=memm_predict_word_tag_pos(validation_data,emi_prob,classifier_word_tag_pos)
weighted_accuracy_wtp = accuracy(memm_prediction_word_tag_pos, validation_labels)
weighted_accuracy_wtp

0.5

In [ ]:
#current word and part of speech
memm_prediction2=memm_predict_word_pos(validation_data,emi_prob,classifier_word_pos)
weighted_accuracy2 = accuracy(memm_prediction2, validation_labels)
weighted_accuracy2

0.5

In [ ]:
#current word and part of speech matrix
memm_true_pos,memm_false_neg,memm_true_neg,memm_false_pos=accuracy_df(memm_prediction2,validation_labels)
memm_df=pd.DataFrame([[memm_true_neg,memm_false_neg], [memm_false_pos, memm_true_pos]],
                   columns=['non-propaganda', 'propaganda'])
memm_df

,non-propaganda,propaganda
0,21029,2113
1,0,0


In [ ]:
#current word, previous word and part of speech
memm_prediction3=memm_predict_cur_pre_word_pos(validation_data,emi_prob,classifier_cur_pre_word_pos)
weighted_accuracy3 = accuracy(memm_prediction3, validation_labels)
weighted_accuracy3

0.5027138620934465

In [ ]:
#current word, previous word and part of speech
memm_true_pos,memm_false_neg,memm_true_neg,memm_false_pos=accuracy_df(memm_prediction3,validation_labels)
memm_df=pd.DataFrame([[memm_true_neg,memm_false_neg], [memm_false_pos, memm_true_pos]],
                   columns=['non-propaganda', 'propaganda'])
memm_df

,non-propaganda,propaganda
0,20964,2095
1,65,18


In [ ]:
#current word, previous word, previous tag and part of speech
memm_prediction_all=memm_predict_all(validation_data,emi_prob,classifier_all)
weighted_accuracy_all = accuracy(memm_prediction_all, validation_labels)
weighted_accuracy_all

0.5084824155009882

In [ ]:
#current word, previous word, previous tag and part of speech
memm_true_pos,memm_false_neg,memm_true_neg,memm_false_pos=accuracy_df(memm_prediction_all,validation_labels)
memm_df=pd.DataFrame([[memm_true_neg,memm_false_neg], [memm_false_pos, memm_true_pos]],
                   columns=['non-propaganda', 'propaganda'])
memm_df

,non-propaganda,propaganda
0,20918,2066
1,111,47


In [ ]:
memm_percent_true_pos=memm_true_pos/(memm_false_neg+memm_true_pos)
memm_percent_true_neg=hmm_true_neg/(memm_false_pos+memm_true_neg)
print("Percentage of propaganda correctly labeled:",memm_percent_true_pos)
print("Percentage of nonpropaganda correctly labeled:",memm_percent_true_neg)

Percentage of propaganda correctly labeled: 0.022243256034074774
Percentage of nonpropaganda correctly labeled: 0.9769366113462361


In [ ]:
#a function that takes in a 1-d list of tokens to output a 1-d list of labels for the submission generater
def memm(data):
  test=[]
  test.append(data)
  tran_prob, emi_prob, init_prob, unk_prob = hmm_train(total_data, total_labels, 0.01)
  memm_prediction = memm_predict_all(test, emi_prob, classifier_all)
  label_sequence=memm_prediction[0].tolist()
  return label_sequence

### Q4: Implementation Details

Q4.1: Explain here how you implemented the MEMM (e.g. which algorithms/data structures you used, what features are included). Make clear which parts were implemented from scratch vs. obtained via an existing package. 


**Answer:** We used the nltk MaxEnt classifier that takes in different features and produces probability of the token having tag '0' or tag '1'. We then used the probability produced by the classifier to replace (emission probability * transition probability) in HMM, and adpated the Viterbi function in HMM.

Q4.2: What features are considered most important by your MaxEnt Classifier? Why do you think these features make sense? Describe your experiments with feature sets.

**Answer:** A combination of current word, previous word, previous tag and part of speech performed the best. It makes sense because this combination provides more context of a word and its function in the sentence. We have experimented with: 

current word + previous tag

current word + previous tag + part of speech

current word + part of speech

current word + previous word + part of speech

current word + previous word + part of speech + previous tag




### Q5: Results Analysis

Q5.1: Explain here how you evaluated the MEMM model. Summarize the performance of your system and any variations that you experimented with the validation datasets. Put the results into clearly labeled tables or diagrams and include your observations and analysis. 

**Answer:** We used different featuresets to train the MaxEnt classifier and applied it to predict labels of the same validation set. We evaluted the performances of different feature sets by calculating the weighted accuracy.

![accuracies](https://drive.google.com/uc?id=1kneTKUwXSwyNb0Kqam8MfXMJ7m4nvZUB)

Q5.2: An analysis on feature selection for the MEMM is required – e.g. what features **help most**, why? An **error analysis** is required – e.g. what sorts of errors occurred, why? 

**Answer:** As the table shows, the first three featuresets have a weighted accuracy of 0.5, and we found that the predictor tagged every token to be nonpropaganda/0. When we added the previous word feature, the predictor started to assign propaganda/1 tags. When we added all 4 features together we reached highest weighted accuracy. It implies that previous word is the most helpful feature, and its combination with part of speech and previous tag made these features more informational than when they are by themselves.

The main error of our MEMM model is that it is prone to assign 0 tags, which may be caused by the fact that 0 tags appear much more often than 1 tags, which makes the classifer more likely to assign 0 tags over 1.

Q5.3: When did the system work well, when did it fail and any ideas as to why? How might you improve the system?

**Answer:** The system has high accuracy in identifying nonpropagandistic words, but fails to identify propagandistic words. One way to improve the system is adding more features to capture more precise context of a given word.




#Comparing HMMs and MEMMs

---


### Q6:


Compare here your results for your HMM and the MEMM. Which of them performs better? 

**Answer:** From the accuracy we can find that HMM perform better. Because the Label Bias Problem, MEMM are less likely to transform the state. Because the propagandistic words are more distributed as a span of words, so the transfrom of state are very uncommon in the training data. MEMM are more likely to predict all 0s because of the Label Bias Problem and the accuracy are worse than HMM. 


###Q7: Error Analysis
Q7.1: Do some error analysis. What are error patterns you observed that the HMM makes but the MEMM does not? Try to justify why/why not?

**Answer:** Based on the example of HMM and MEMM, we can find that because of the transition probility and the propagandistic words ususally are distributed as a span of words. HMM are more likely to predict consecutive words as propagandistic words. We use capitalize to indicate propagandistic words. Such as in ground truth, <br>**BOTCHED INVESTIGATION is a stunning indictment of the fbi**.<br> And in HMM, <br>**BOTCHED INVESTIGATION IS A STUNNING INDICTMENT OF THE FBI**.<br> And in MEMM, <br>**botched INVESTIGATION is a stunning INDICTMENT of the fbi**.<br> We can see that HMM are predict the whole sentence as propagandistic and MEMM only predict few words. 

Q7.2: What are error patterns you observed that MEMM makes but the HMM does not? Try to justify what you observe?

**Answer:** Generally speaking, HMM are more likely to predict as propagandistic and MEMM are not. So most of errors are occur in MEMM and not in HMM. This might because we use add-k smoothing in HMM and it will more likely to classify a word to propagandistic and MEMM are not. Another reason is because of Label Bias Problem, MEMM are more likely to stay on state 0. Such as in ground truth <br>**LOCAL AND FEDERAL AUTHORITIES ARE REFUSING TO FILL IN THE BLANKS**<br> In HMM, <br>**LOCAL AND FEDERAL AUTHORITIES ARE REFUSING TO FILL IN THE BLANKS**<br> And in MEMM <br> **local and federal authorities are refusing to fill in the blanks**<br> The whole scentence are missing because MEMM are more likely to stay on state 0, which is non-propagandistic.




In [ ]:
print("All propagandistic words are capitalized")
print("Use one article from validation set as example")
example = 4
single_data = validation_data[example]
print("Ground Truth")
single_label = validation_labels[example]
article = ''
for i in range(len(single_data)):
  word = single_data[i]
  if single_label[i]:
    word = word.upper()
  article += word + ' '
print(article)
print()

print("HMM prediction")
single_label = hmm_prediction[example]
article = ''
for i in range(len(single_data)):
  word = single_data[i]
  if single_label[i]:
    word = word.upper()
  article += word + ' '
print(article)
print()

print("MEMM prediction")
single_label = memm_prediction_all[example]
article = ''
for i in range(len(single_data)):
  word = single_data[i]
  if single_label[i]:
    word = word.upper()
  article += word + ' '
print(article)
print()

All propagandistic words are capitalized
Use one article from validation set as example
Ground Truth
local police & feds impose information blackout in las vegas shooting there already was a blackout on the worst mass murder in modern american history . 58 dead , hundreds wounded , the savagery of the vegas attack is unspeakable , and STILL THE INCOMPETENT , CLUELESS AUTHORITIES KNOW NOTHING . THAT ’ S THE INFORMATION THEY SEEK TO BLACKOUT . this BOTCHED INVESTIGATION is a stunning indictment of the fbi . THIS IS NOT THE SOVIET UNION , THIS IS NOT IRAN OR RIYADH – THIS IS AMERICA . where is the outrage ? and while the BUMBLING FBI has dismissed out of hand jihad as a motive , here is what we know : isis has claimed responsibility for the vegas slaughter . isis does not claim credit for events that are not theirs . they have double and tripled down on their claim . they have said that stephen paddock had converted to islam six months ago , and they revealed his islamic name as abu abd a

# Kaggle Submission 
---

Using the best-performing system from among all of your HMM and MEMM models, generate predictions for the test set, and submit them to Kaggle at https://www.kaggle.com/t/8a8030baefcc4d91b715f114353dba38. Note, you **need** to use our tokenizer as the labels on Kaggle corresponds to these. Below, we provide a script that submits a random guess/all ones or all zeroes in the correct format. Note that you only need to provide a function which takes as input a sequence of tokens, and outputs a sequence of labels (in the form of integers  in {0,1}, 1 corresponding to propaganda). As a scoring metric on Kaggle, we use **weighted accuracy**, described in the *Notes* section towards the beginning of the notebook.


In [ ]:
import random
from typing import Callable


def submission_to_csv(binary_class : List[List[int]], fname : str):
  # writes submission to CSV
  new_binary = []
  for classes in binary_class:
    new_binary += classes
  with open(fname, 'w') as file:
    file.write('id,category\n')
    for i, line in enumerate(new_binary):
      file.write('{},{}\n'.format(i,line))


def random_predictor(article_tokens : List[str]) -> List[int]:
  # random predictor
  output = []
  for _ in range(len(article_tokens)):
    output.append(random.randint(0,1))
  return output

def always_one(article_tokens : List[str]) -> List[int]:
  # random predictor
  output = []
  for _ in range(len(article_tokens)):
    output.append(1)
  return output

def always_zero(article_tokens : List[str]) -> List[int]:
  # random predictor
  output = []
  for _ in range(len(article_tokens)):
    output.append(0)
  return output


def generate_submission(predictor: Callable[[List[str]], List[int]], fname : str, test_path : str):
  # generate a submission with random guesses
  sample_submission = []
  articles = os.listdir(test_path)
  total, fail, targets, test_articles = 0,0, [], []
  for article in sorted(articles):
    article_file = os.path.join(test_path, article)
    art_toks = tokenize_article(article_file)
    curr_article = predictor(art_toks)
    sample_submission.append(curr_article)
  submission_to_csv(sample_submission, fname)


test_path=os.path.join(os.getcwd(), "drive", "My Drive/CS4740/project2","test")
generate_submission(random_predictor, "random_submission.csv", test_path)
generate_submission(always_one, "ones_submission.csv", test_path)
generate_submission(always_zero, "zeros_submission.csv", test_path)


In [ ]:
generate_submission(hmm, "hmm_submission2.csv", test_path)

In [ ]:
generate_submission(memm, "memm_submission3.csv", test_path)

---
### Q8: Competition Score

Include here your **team name** and the **screenshot** of your best score from Kaggle.

**Answer:** team name: yl2546_zz727

![kaggle](https://drive.google.com/uc?id=1RrGS9b36itgnUR2YVH3Wwgtryjh6iP1U)

In [1]:
%%capture
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

In [401]:
#convert to pdf
%%capture
# the red text is a placeholder! Change it to your directory structure!
!cp 'drive/My Drive/Colab Notebooks/4740_FA20_p2_yl2546_zz727.ipynb' ./ 

In [403]:
# the red text is a placeholder! Change it to the name of this notebook!
# !jupyter nbconvert --to PDF "4740_FA20_p2_yl2546_zz727.ipynb"

#Appendix
---

### Tentative grading guideline

- (5 pts) Dataset exploration.
- (20 pts) Design and implementation of the HMM model. Specifically, calculations of emission/transition probability, smoothing, handling of unknown words.
- (20 pts) Design and implementation of the MEMM, as well as the feature set and how the feature extraction is done.
- (20 pts) Experiment design and methodology. Investigations into feature sets and unknown word handling/smoothing.
- (20 pts) Error analysis and comparison of **HMM** model with the **MEMM**. 
- (10 pts) Report: organization, clarity and including the **corresponding pieces of code for the implementations**.
- (5 pts) **Submission to Kaggle**. In the report, you should add a screenshot of your team’s performance on kaggle leaderboard.

